In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm, tqdm_notebook,tqdm_pandas
import itertools
import re
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
filter_word = pd.read_excel('filter_word.xlsx')
filter_word = filter_word['filter_word'].values

### 분할이 된 파일들만 
분할이 된 파일들만 통합하고 각 폴더밖으로 꺼내준다 

In [3]:
# path = './output/token/'
path = './output/token/'
file_list = os.listdir(path)
folder_list = []
for file in file_list:
    if os.path.splitext(file)[1] != '.csv':
        folder_list.append(file)

for folder in tqdm_notebook(folder_list):
    file_list = os.listdir(f'{path}/{folder}/')
    all_df = pd.DataFrame()
    for file in file_list:
        read_file = pd.read_csv(f'{path}/{folder}/{file}')
        all_df = pd.concat([all_df,read_file])
        size = all_df.shape[0]
    all_df.to_csv(f'./output/token/{folder}_0~{size}.csv',index=False)


### 파일결합
* 검색어를 다수로 사용했기에 각 파일을 마을별로 묶어준다

In [4]:
path = './output/token/'
file_list = os.listdir(path)
csv_list = []
for file in file_list:
    if os.path.splitext(file)[1] == '.csv':
        csv_list.append(file)

In [5]:
merge_file = dict()

merge_file['광진'] = ['광진리','광진','광진해변','휴휴암'] #광진 +강원도 삭제
merge_file['기사문'] = ['기사문리','기사문','기사문항','기사문해변']
merge_file['낙산'] = ['낙산','낙산항','낙산해수욕장']
merge_file['남애'] = ['남애리','남애','남애항'] #남애 +강원도 삭제
merge_file['동호'] = ['동호','동호리','동호해변','동호해수욕장']
merge_file['동산'] = ['동산','동산리','동산항','동산해수욕장'] #동산 _강원도 삭제
merge_file['물치'] = ['물치리','물치','물치항']
merge_file['수산'] = ['수산리','수산항'] #'수산' 제외
merge_file['오산'] = ['오산','오산해수욕장','오산리'] #오산 +강원도 삭제
merge_file['인구'] = ['인구리','죽도'] # '인구' 제외
merge_file['전진'] = ['전진리','설악해수욕장','후진항']
merge_file['하광정'] = ['하광정리','하조대','하조대해수욕장']
merge_file['강원도'] = ['강원도']
merge_file['속초'] = ['속초']
merge_file['강릉'] = ['강릉']
merge_file['양양'] = ['양양']

In [6]:
#결합해야하는 파일 솎아내기
files = dict()
for file in csv_list:
    keyword = file.split('_')[0].split("+")[0].replace(' ', '')
    for key in merge_file.keys():
        if keyword in merge_file[key]:
            if files.get(key) == None:
                files[key] = [file]
            else:
                files[key].append(file)
    
#파일결합
for key in tqdm_notebook(files.keys()):
    merged_df = pd.DataFrame()
    for file in files[key]:
        file_df = pd.read_csv(path + file)
        merged_df = pd.concat([merged_df,file_df])
    merged_df = merged_df.drop_duplicates('full_text')
    
    #필터링
    merged_df = merged_df.fillna('')
    filtered = merged_df.full_text.apply(lambda x : any(ele in x for ele in filter_word)) #전체글
    clean_file = merged_df[~filtered]
    filtered = clean_file.title.apply(lambda x : any(ele in x for ele in filter_word)) #타이틀
    clean_file = clean_file[~filtered]
    
    total_len = clean_file.shape[0]
    clean_file.to_csv(f'./output/token_통합/{key}_{total_len}.csv',index = False)

### 파일 종합 결과

In [7]:
path = './output/token_통합/'
file_list = os.listdir(path)

total_df = pd.DataFrame()
for file in file_list:
    sumup = dict()
    keyword = file.split('_')[0]
    df = pd.read_csv(path +file)
    sumup['total_size'] = df.shape[0]
    for year in range(2010,2020):
        try :
            sumup[year] = df[df.year == year].shape[0]
        except :
            sumup[year] = np.nan
    one_df = pd.DataFrame(sumup,index =[keyword])
    total_df = pd.concat([total_df,one_df])

In [8]:
total_df

,total_size,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
강릉,51365,2650,3405,4230,4738,5607,6146,6102,6062,6157,6268
강원도,59239,5139,5468,5654,5824,6236,6700,6427,6443,5868,5480
광진,5841,116,161,234,380,530,709,886,1043,843,939
기사문,2516,29,59,79,132,251,356,355,374,389,492
낙산,19600,627,840,865,1048,1488,2055,2348,2760,2951,4618
남애,4348,108,118,171,227,379,534,604,801,741,665
동산,1970,48,69,77,92,134,199,190,255,446,460
동호,3357,89,110,119,159,235,420,419,577,608,621
물치,4313,144,143,207,234,305,412,492,682,870,824
속초,55652,2475,3498,4361,5088,6345,6944,6858,6832,6615,6636


### 각각탐색

In [9]:
path = './output/token_통합/'
file_list = os.listdir(path)

In [10]:
file_list

['강릉_51365.csv',
 '강원도_59239.csv',
 '광진_5841.csv',
 '기사문_2516.csv',
 '낙산_19600.csv',
 '남애_4348.csv',
 '동산_1970.csv',
 '동호_3357.csv',
 '물치_4313.csv',
 '속초_55652.csv',
 '수산_2529.csv',
 '양양_46106.csv',
 '오산_3248.csv',
 '인구_8401.csv',
 '전진_2447.csv',
 '하광정_16616.csv']

In [11]:
sample = pd.read_csv(path +'동산_8917.csv')

FileNotFoundError: [Errno 2] File b'./output/token_\xed\x86\xb5\xed\x95\xa9/\xeb\x8f\x99\xec\x82\xb0_8917.csv' does not exist: b'./output/token_\xed\x86\xb5\xed\x95\xa9/\xeb\x8f\x99\xec\x82\xb0_8917.csv'

In [ ]:
sample.title.value_counts().head(20)

In [ ]:
sample.fillna("",inplace = True)
sample[sample.Noun.apply(lambda x : "초경" in x)].year.value_counts()